# Credit Risk Resampling Techniques

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [22]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [30]:
# Load the data
file_path = Path('dummy_data_3.csv')
df = pd.read_csv(file_path, skiprows=1)
df = df.loc[:, columns].copy()

# # # Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# # # Drop the null rows
df = df.dropna()

# # # Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# # # convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# # Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

# x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(30)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10000,0.0646,306.31,RENT,75000,Source Verified,Feb-2019,low_risk,n,25.01,...,100.0,80.0,1,0,85879,63930,28900,47279,N,N
1,40000,0.1102,1309.93,RENT,130000,Source Verified,Feb-2019,low_risk,n,15.42,...,94.1,80.0,0,0,70202,44757,40500,24202,N,N
2,1600,0.0881,50.74,RENT,89000,Source Verified,Feb-2019,low_risk,n,10.14,...,92.9,20.0,0,0,50513,31629,13100,31713,N,N
3,25000,0.1171,826.90,RENT,68000,Source Verified,Feb-2019,In Grace Period,n,27.02,...,100.0,0.0,0,0,98784,63326,29100,58984,N,N
4,12000,0.0819,377.09,MORTGAGE,56000,Verified,Feb-2019,low_risk,n,10.76,...,100.0,36.4,0,0,403515,22657,21100,7500,N,N
5,24000,0.1614,585.43,RENT,90000,Source Verified,Feb-2019,low_risk,n,15.49,...,88.9,66.7,0,0,302468,38884,32000,28668,N,N
6,15000,0.1171,496.14,MORTGAGE,40000,Verified,Feb-2019,low_risk,n,40.08,...,100.0,14.3,0,0,126807,63910,30400,83607,N,N
7,12000,0.1390,278.60,MORTGAGE,74500,Not Verified,Feb-2019,low_risk,n,10.13,...,95.2,11.1,0,0,27881,10910,6400,13781,N,N
8,25000,0.0819,785.61,MORTGAGE,48000,Not Verified,Feb-2019,low_risk,n,24.45,...,94.1,100.0,0,0,220407,43788,9500,50940,N,N
9,14000,0.1102,458.48,RENT,48000,Not Verified,Feb-2019,low_risk,n,17.39,...,96.9,50.0,0,0,98122,83900,9500,77722,N,N


# Split the Data into Training and Testing

In [31]:
# Change strings to numeric values
encoded_df = pd.get_dummies(df, columns = ["home_ownership", 
                                           "verification_status", 
                                           "pymnt_plan", 
                                           "initial_list_status", 
                                           "application_type",
                                           "hardship_flag",
                                           "debt_settlement_flag",
                                           "issue_d",
                                           "next_pymnt_d"])
# # Change dates to numeric
# # Code inspired by: https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas
# import datetime

# encoded_df['issue_d_year'] = pd.DatetimeIndex(encoded_df['issue_d']).year
# encoded_df['issue_d_month'] = pd.DatetimeIndex(encoded_df['issue_d']).month

# encoded_df['next_pymnt_d_year'] = pd.DatetimeIndex(encoded_df['next_pymnt_d']).year
# encoded_df['next_pymnt_d_month'] = pd.DatetimeIndex(encoded_df['next_pymnt_d']).month


# Create our features
X = encoded_df.copy()
X = X.drop(['loan_status'], axis = 1)


# Create our target
y = encoded_df['loan_status'].values




In [32]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,issue_d_Feb-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
count,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,...,17.000000,17.000000,17.000000,17.000000,17.0,17.0,17.000000,17.000000,17.000000,17.000000
mean,18261.764706,0.121706,549.454706,75411.764706,22.271765,0.058824,0.294118,14.117647,0.235294,14240.705882,...,0.117647,0.882353,0.705882,0.294118,1.0,1.0,0.941176,0.058824,0.823529,0.176471
std,11988.070625,0.052915,351.085378,29115.736346,12.237994,0.242536,0.771744,6.142906,0.437237,11131.434632,...,0.332106,0.332106,0.469668,0.469668,0.0,0.0,0.242536,0.242536,0.392953,0.392953
min,1600.000000,0.064600,50.740000,36000.000000,10.130000,0.000000,0.000000,5.000000,0.000000,424.000000,...,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000
25%,10000.000000,0.081900,278.600000,50000.000000,15.420000,0.000000,0.000000,10.000000,0.000000,5087.000000,...,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.000000,0.000000,1.000000,0.000000
50%,14000.000000,0.110200,458.480000,72500.000000,18.430000,0.000000,0.000000,13.000000,0.000000,14569.000000,...,0.000000,1.000000,1.000000,0.000000,1.0,1.0,1.000000,0.000000,1.000000,0.000000
75%,25000.000000,0.129800,785.610000,90000.000000,25.010000,0.000000,0.000000,16.000000,0.000000,18481.000000,...,0.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,0.000000,1.000000,0.000000
max,40000.000000,0.290000,1309.930000,130000.000000,58.790000,1.000000,3.000000,27.000000,1.000000,37102.000000,...,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000


In [33]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 16, 'In Grace Period': 1})

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [ ]:
# Resample the training data with the RandomOversampler
print("Balance before resampling (training data): ")
print(Counter(y_train))

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print("Balance after resampling: ")
print(Counter(y_resampled))

Balance before resampling (training data): 
Counter({'low_risk': 12})


ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [10]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state = 1)

model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)


In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
# bal_acc_score = balanced_accuracy_score(y_test, y_pred)
# print(f'Balanced accuracy score: {bal_acc_score}')

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[   57    30]
 [ 5801 11317]]


In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
results = classification_report_imbalanced(y_test, y_pred)
print(results)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.66      0.02      0.66      0.43        87
   low_risk       1.00      0.66      0.66      0.80      0.66      0.43     17118

avg / total       0.99      0.66      0.66      0.79      0.66      0.43     17205



### SMOTE Oversampling

In [14]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE(random_state = 1, sampling_strategy = 'auto').fit_resample(X_train,y_train)

Counter(y_resampled_smote)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [15]:
# Train the Logistic Regression model using the resampled data
smote_model = LogisticRegression(random_state = 1)
smote_model.fit(X_resampled_smote, y_resampled_smote)

y_pred_smote = smote_model.predict(X_test)


In [16]:
# Calculated the balanced accuracy score
smote_bal_accuracy = balanced_accuracy_score(y_test, y_pred_smote)
print(smote_bal_accuracy)

0.6467380575397546


In [17]:
# Display the confusion matrix
smote_cm = confusion_matrix(y_test, y_pred_smote)
print(smote_cm)


[[   55    32]
 [ 5798 11320]]


In [18]:
# Print the imbalanced classification report
smote_results = classification_report_imbalanced(y_test, y_pred_smote)
print(smote_results)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.66      0.02      0.65      0.42        87
   low_risk       1.00      0.66      0.63      0.80      0.65      0.42     17118

avg / total       0.99      0.66      0.63      0.79      0.65      0.42     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [19]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state = 1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train, y_train)
Counter(y_resampled_cc)



there are negative numbers in 'values'
the negative numbers are: [-1.734723475976807e-18, -6.938893903907228e-18, -5.204170427930421e-18, -1.734723475976807e-18, -1.734723475976807e-18, -5.204170427930421e-18, -1.734723475976807e-18, -1.734723475976807e-18, -1.734723475976807e-18, -1.734723475976807e-18, -1.734723475976807e-18, -6.938893903907228e-18, -1.734723475976807e-18, -1.734723475976807e-18, -1.734723475976807e-18, -5.204170427930421e-18, -1.734723475976807e-18, -1.734723475976807e-18, -1.734723475976807e-18, -6.938893903907228e-18, -6.938893903907228e-18, -1.734723475976807e-18, -5.204170427930421e-18, -5.204170427930421e-18, -3.469446951953614e-18, -6.938893903907228e-18, -3.469446951953614e-18, -1.734723475976807e-18, -3.469446951953614e-18, -6.938893903907228e-18, -3.469446951953614e-18, -1.734723475976807e-18, -3.469446951953614e-18, -3.469446951953614e-18, -5.204170427930421e-18, -5.204170427930421e-18, -1.734723475976807e-18]


ValueError: Cannot losslessly cast from float64 to uint8

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
smote_results = classification_report_imbalanced(y_test, y_pred_smote)
print(smote_results)

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [20]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state = 1)
X_resampled_smote_enn, y_resampled_smote_enn = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled_smote_enn)

Counter({'high_risk': 51351, 'low_risk': 46389})

In [21]:
# Train the Logistic Regression model using the resampled data
smote_enn_model = LogisticRegression(random_state=1)
smote_enn_model.fit(X_resampled_smote_enn, y_resampled_smote_enn)
y_pred_smote_enn = smote_enn_model.predict(X_test)

In [22]:
# Calculated the balanced accuracy score
smote_enn_bal_acc_score = balanced_accuracy_score(y_test, y_pred_smote_enn)
print(smote_enn_bal_acc_score)

0.6107263578165351


In [23]:
# Display the confusion matrix
cm_smote_enn = confusion_matrix(y_test, y_pred_smote_enn)
print(cm_smote_enn)

[[  67   20]
 [9392 7726]]


In [24]:
# Print the imbalanced classification report
smote_enn_results = classification_report_imbalanced(y_test, y_pred_smote_enn)
print(smote_enn_results)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.77      0.45      0.01      0.59      0.36        87
   low_risk       1.00      0.45      0.77      0.62      0.59      0.34     17118

avg / total       0.99      0.45      0.77      0.62      0.59      0.34     17205

